In [2]:
import pandas as pd
import jinja2
from collections import OrderedDict
from json import dumps
from IPython.html.widgets import interact
from IPython.html import widgets
from IPython.display import display, display_pretty, Javascript, HTML
from IPython.utils.traitlets import Any, Bool, Dict, List, Unicode

gene_pheno = pd.read_csv(open("genepheno_exp.csv"))
gene_pheno['value'] = 1

def query(query_input, pheno_input):
    gene_list = []
    pheno_list = []
    if pheno_input != '':
        pheno_list = pheno_input.split(',')
        pheno_list = [x.strip(' ') for x in pheno_list]
    if gene_input != '':
        gene_list = gene_input.split(',')
        gene_list = [x.strip(' ') for x in gene_list]
    gene_list_ext = []
    if len(gene_list) > 0 and len(pheno_list) > 0:
        gene_list_ext = list(gene_pheno[gene_pheno.pheno.isin(pheno_list) & gene_pheno.gene.isin(gene_list)]['gene'].value_counts()[:50].keys())
        final_df = gene_pheno[gene_pheno.pheno.isin(pheno_list) & gene_pheno.gene.isin(gene_list_ext)]
    elif len(gene_list) > 0 and len(pheno_list) == 0: 
        pheno_list_ext = list(gene_pheno[gene_pheno.gene.isin(gene_list)]['pheno'].value_counts()[:50].keys())
        final_df = gene_pheno[gene_pheno.pheno.isin(pheno_list_ext) & gene_pheno.gene.isin(gene_list)]
    elif len(gene_list) ==0 and len(pheno_list) > 0:
        gene_list_ext = list(gene_pheno[gene_pheno.pheno.isin(pheno_list)]['gene'].value_counts()[:50].keys())
        final_df = gene_pheno[gene_pheno.pheno.isin(pheno_list) & gene_pheno.gene.isin(gene_list_ext)]
    else:
        final_df = "Invalid input!"
    return final_df

def printResults(query_res):
    gene_counts = query_res['gene'].value_counts()
    gene_count_df = gene_counts.to_frame('phenos')
    result = query_res.join(gene_count_df, on='gene')
    print "GENE\t\t\t\tPHENO\t\t\t\t\tTotal Phenos\n"
    for row in result.sort_values(by=['phenos','gene'], ascending=[0,1]).itertuples():
        print row[1]+"\t\t\t\t"+row[2]+"\t\t\t\t"+'%.2f' % row[4]
        
def plotOut():
    display(Javascript(bipartite_template.render(
    data=query_res.itertuples())))
    return


In [3]:
%%javascript
require.config({
    paths: {
        d3: 'http://d3js.org/d3.v3.min'
    }
});

<IPython.core.display.Javascript object>

In [8]:
bipartite_template = jinja2.Template(
"""
require(["d3"], function(d3) {
    var raw_input = []

    {% for row in data %}
    raw_input.push(['{{ row[1] }}', '{{ row[2] }}', {{ row[3] }}]);
    {% endfor %}
    
    d3.select("#chart_d3 svg").remove()
    
    var width = 1100, height = 980, margin ={b:0, t:40, l:170, r:50};

    var svg = d3.select("#chart_d3")
        .append("svg").attr('width',width).attr('height',(height+margin.b+margin.t))
        .append("g").attr("transform","translate("+ margin.l+","+margin.t+")");

    var data = [ 
        {data:bP.partData(raw_input,2), id:'GenePheno', header:["Gene","Pheno", "Interactions"]}
    ];
    
    bP.draw(data, svg);
});
"""
)

In [9]:
from IPython.display import HTML

input_form = """
<!DOCTYPE html>
<meta charset="utf-8">
<style>
body{
    width:1200px;
    height:1200px;
    margin:100px;
}
svg text{
    font-size:16px;
}
rect{
    shape-rendering:crispEdges;
}
</style>
<body>
<script src="biPartite.js"></script>
<br /><br /><br /><br /><br />
<div style="background-color:gainsboro; border:solid black; width:860px; padding:20px;">
<b>Search for Gene-Pheno Interactions</b>
<div>
Genes: <input style='width:100%' type="text" id="gene_input" value="ASPM, ASXL2, BAZ1B, DEAF1, DNM1, ARHGEF9, RAI1"><br>
Phenotypes: <input style='width:100%' type="text" id="pheno_input" value="behavioral abnormality, speech impairment, mental retardation"><br>
<button onclick="search()">Search Interactions</button>
</div>
</div>
<div style="height:100%" id="chart_d3"/>
</body>
"""
javascript = """
<script type="text/Javascript">
    function search(){
        var gene_input_value = document.getElementById('gene_input').value;
        var pheno_input_value = document.getElementById('pheno_input').value;
        var command_gene = "gene_input = '" + gene_input_value + "'";
        var command_pheno = "pheno_input = '" + pheno_input_value + "'";
        var command_query = "query_res = query(gene_input,pheno_input);";
        var kernel = IPython.notebook.kernel;
        kernel.execute(command_gene);
        kernel.execute(command_pheno);
        kernel.execute(command_query);
    }
</script>
"""
HTML(input_form + javascript)

In [10]:
printResults(query_res)

AttributeError: 'Series' object has no attribute 'to_frame'

In [11]:
display(Javascript(bipartite_template.render(
    data=query_res.itertuples())))

<IPython.core.display.Javascript object>